In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.mixture import BayesianGaussianMixture
from sklearn import model_selection

import random
import math
import datetime

In [ ]:
df = pd.read_csv("archive/User0_credit_card_transactions.csv")
# print(df.to_string())
# print(df.columns)

df_copy = df.copy() #IDK if we'll need this later

df = df.drop(columns=["Is Fraud?"])
data = df.to_numpy()

# print(df.to_string())
# print(data[0, :])

In [ ]:
#need to convert the CSV text values into numbers

num_transaction_types = 0
num_merchant_cities = 0
num_merchant_states = 0
num_errors = 0

transaction_types = {}
merchant_cities = {}
merchant_states = {}
errors = {}

zero_time = datetime.datetime.strptime("00:00", "%H:%M")

X = np.ndarray(data.shape, dtype=float)

for i in range(data.shape[0]):
    # print(data[i, 5])

    #these values are already numbers; no need to change them
    #TODO: normalize some of them?
    X[i, 0:5] = data[i, 0:5]
    X[i, 8] = data[i, 8]
    X[i, 11:13] = data[i, 11:13]

    #convert the time of date string into the number of minutes past midnight
    dtime = datetime.datetime.strptime(data[i, 5], "%H:%M")
    X[i, 5] = (dtime - zero_time).seconds//60

    #convert dollar amount from "$134.03" to 134.03
    X[i, 6] = float(data[i, 6][1:])

    #use ints to represent the different transaction types
    if not data[i, 7] in transaction_types:
        transaction_types[data[i, 7]] = num_transaction_types
        num_transaction_types += 1
    X[i, 7] = transaction_types[data[i, 7]]

    #use ints to represent the different cities
    if not data[i, 9] in merchant_cities:
        merchant_cities[data[i, 9]] = num_merchant_cities
        num_merchant_cities += 1
    X[i, 9] = merchant_cities[data[i, 9]]

    #use ints to represent the different states
    if not data[i, 10] in merchant_states:
        merchant_states[data[i, 10]] = num_merchant_states
        num_merchant_states += 1
    X[i, 10] = merchant_states[data[i, 10]]

    #use ints to represent errors
    if not data[i, 13] in errors:
        errors[data[i, 13]] = num_errors
        num_errors += 1
    X[i, 13] = errors[data[i, 13]]

    for j in range(X.shape[1]):
        if math.isnan(X[i, j]): X[i, j] = 0

    # print(data[i])    
    # print(X[i], X[i, 11], data[i, 11])

In [ ]:
X_train, X_test = model_selection.train_test_split(X, test_size=0.2)

In [ ]:
bgm = BayesianGaussianMixture()
bgm.fit(X_train)

probs = bgm.predict_proba(X_test)
print(probs[probs != 1]) #note: this currently only returns 1.0 for all samples (the mixture must be tuned)